In [1]:
#! pip install -U -q tf-models-official

In [4]:
import orbit
import tensorflow_models as tfm
import tensorflow as tf
import os
from official.core import exp_factory
from official.core import config_definitions as cfg
from official.vision.serving import export_saved_model_lib
from official.vision.ops.preprocess_ops import normalize_image
from official.vision.ops.preprocess_ops import resize_and_crop_image
from official.vision.utils.object_detection import visualization_utils
from official.vision.dataloaders.tf_example_decoder import TfExampleDecoder

#pp = pprint.PrettyPrinter(indent=4) # Set Pretty Print Indentation
print(tf.__version__) # Check the version of tensorflow used

2.10.1


In [74]:
from official.vision.modeling.factory import detection_generator

detection_generator

In [24]:
#pipeline_config = os.path.join('models/research/object_detection/configs/tf2/',
#                                model_name + '.config')
#model_dir = 'models/research/object_detection/test_data/checkpoint/'
pipeline_config = "models/coco_spinenet49_mobile_tpu.yaml"

# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(model_yaml["task"]["model"])
model_config = configs['model']
detection_model = model_builder.build(
      model_config=model_config, is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(
      model=detection_model)
ckpt.restore(os.path.join(model_dir, 'ckpt-0')).expect_partial()

NameError: name 'config_util' is not defined

In [148]:
from tensorflow.keras.models import model_from_json
import yaml
# Load YAML file
with tf.io.gfile.GFile(pipeline_config, 'r') as yaml_file:
    yaml_string = yaml_file.read()

# Parse YAML string
model_yaml = yaml.safe_load(yaml_string)


In [16]:
from official.vision.modeling.backbones import resnet



In [17]:
input_size = 128
filter_size_scale, block_repeats, resample_alpha, endpoints_num_filters, min_level, max_level = 0.65, 1, 0.5, 128, 4, 6
input_specs = tf.keras.layers.InputSpec(
    shape=[None, input_size, input_size, 3])
model = tfm.vision.backbones.SpineNet(
    input_specs=input_specs,
    min_level=min_level,
    max_level=max_level,
    endpoints_num_filters=endpoints_num_filters,
    resample_alpha=resample_alpha,
    block_repeats=block_repeats,
    filter_size_scale=filter_size_scale,
    init_stochastic_depth_rate=0.2,
)

inputs = tf.keras.Input(shape=(input_size, input_size, 3), batch_size=1)
endpoints = model(inputs)
print(model.name)
print(endpoints)

spine_net
{'4': <KerasTensor: shape=(1, 8, 8, 128) dtype=float32 (created by layer 'spine_net')>, '5': <KerasTensor: shape=(1, 4, 4, 128) dtype=float32 (created by layer 'spine_net')>, '6': <KerasTensor: shape=(1, 2, 2, 128) dtype=float32 (created by layer 'spine_net')>}


In [18]:
model.from_config(pipeline_config)

TypeError: official.vision.modeling.backbones.spinenet.SpineNet() argument after ** must be a mapping, not str

In [23]:
model_yaml["task"]["model"]

{'anchor': {'anchor_size': 3,
  'aspect_ratios': [0.5, 1.0, 2.0],
  'num_scales': 3},
 'backbone': {'spinenet_mobile': {'stochastic_depth_drop_rate': 0.2,
   'model_id': '49',
   'se_ratio': 0.2},
  'type': 'spinenet_mobile'},
 'decoder': {'type': 'identity'},
 'head': {'num_convs': 4, 'num_filters': 48, 'use_separable_conv': True},
 'input_size': [384, 384, 3],
 'max_level': 7,
 'min_level': 3,
 'norm_activation': {'activation': 'hard_swish',
  'norm_epsilon': 0.001,
  'norm_momentum': 0.99,
  'use_sync_bn': True}}

In [25]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'    # Suppress TensorFlow logging (1)
import pathlib
import tensorflow as tf

tf.get_logger().setLevel('ERROR')           # Suppress TensorFlow logging (2)

# Enable GPU dynamic memory allocation
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

def download_images():
    base_url = 'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/test_images/'
    filenames = ['image1.jpg', 'image2.jpg']
    image_paths = []
    for filename in filenames:
        image_path = tf.keras.utils.get_file(fname=filename,
                                            origin=base_url + filename,
                                            untar=False)
        image_path = pathlib.Path(image_path)
        image_paths.append(str(image_path))
    return image_paths

IMAGE_PATHS = download_images()

1415684/1415684 [==============================] - 0s 0us/step


In [26]:
# Download and extract model
def download_model(model_name, model_date):
    base_url = 'http://download.tensorflow.org/models/object_detection/tf2/'
    model_file = model_name + '.tar.gz'
    model_dir = tf.keras.utils.get_file(fname=model_name,
                                        origin=base_url + model_date + '/' + model_file,
                                        untar=True)
    return str(model_dir)

MODEL_DATE = '20200711'
MODEL_NAME = 'centernet_hg104_1024x1024_coco17_tpu-32'
PATH_TO_MODEL_DIR = download_model(MODEL_NAME, MODEL_DATE)

1426460092/1426460092 [==============================] - 93s 0us/step


In [27]:
# Download labels file
def download_labels(filename):
    base_url = 'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/data/'
    label_dir = tf.keras.utils.get_file(fname=filename,
                                        origin=base_url + filename,
                                        untar=False)
    label_dir = pathlib.Path(label_dir)
    return str(label_dir)

LABEL_FILENAME = 'mscoco_label_map.pbtxt'
PATH_TO_LABELS = download_labels(LABEL_FILENAME)

5056/5056 [==============================] - 0s 0s/step


In [28]:
import time
from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

PATH_TO_CFG = PATH_TO_MODEL_DIR + "/pipeline.config"
PATH_TO_CKPT = PATH_TO_MODEL_DIR + "/checkpoint"

print('Loading model... ', end='')
start_time = time.time()

# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(PATH_TO_CFG)
model_config = configs['model']
detection_model = model_builder.build(model_config=model_config, is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(PATH_TO_CKPT, 'ckpt-0')).expect_partial()

@tf.function
def detect_fn(image):
    """Detect objects in image."""

    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)

    return detections

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

ModuleNotFoundError: No module named 'object_detection'

In [67]:
model2 = tfm.vision.backbones.SpineNet.from_config(model_yaml)

In [68]:
# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=model2)
ckpt.restore("models/ckpt-277200").expect_partial()

In [69]:
ckpt

In [70]:
ret = model2.predict(np.zeros((1,512,512,3)))

1/1 [==============================] - 2s 2s/step


In [71]:
ret["3"].shape

(1, 64, 64, 256)

In [72]:
ret["7"].shape

(1, 4, 4, 256)

In [73]:
for key, val in ret.items():
    print(val.shape)

(1, 64, 64, 256)
(1, 32, 32, 256)
(1, 16, 16, 256)
(1, 8, 8, 256)
(1, 4, 4, 256)


In [75]:
from official.vision.tasks import RetinaNetTask
from official.vision.configs.common

In [149]:
task = RetinaNetTask(model_yaml["task"])

In [150]:
task.build_model()

AttributeError: 'dict' object has no attribute 'model'

In [79]:
model_yaml

{'runtime': {'distribution_strategy': 'tpu',
  'mixed_precision_dtype': 'bfloat16'},
 'task': {'losses': {'l2_weight_decay': 3e-05},
  'model': {'anchor': {'anchor_size': 3,
    'aspect_ratios': ListWrapper([0.5, 1.0, 2.0]),
    'num_scales': 3},
   'backbone': {'spinenet_mobile': {'stochastic_depth_drop_rate': 0.2,
     'model_id': '49',
     'se_ratio': 0.2},
    'type': 'spinenet_mobile'},
   'decoder': {'type': 'identity'},
   'head': {'num_convs': 4, 'num_filters': 48, 'use_separable_conv': True},
   'input_size': ListWrapper([384, 384, 3]),
   'max_level': 7,
   'min_level': 3,
   'norm_activation': {'activation': 'hard_swish',
    'norm_epsilon': 0.001,
    'norm_momentum': 0.99,
    'use_sync_bn': True}},
  'train_data': {'dtype': 'bfloat16',
   'global_batch_size': 256,
   'is_training': True,
   'parser': {'aug_rand_hflip': True,
    'aug_scale_max': 2.0,
    'aug_scale_min': 0.5}},
  'validation_data': {'dtype': 'bfloat16',
   'global_batch_size': 256,
   'is_training': Fals

In [106]:
from official.core import config_definitions as cfg

In [136]:
cfg.base_config.Config.from_yaml(pipeline_config)

KeyError: "The key 'runtime' does not exist in <class 'official.modeling.hyperparams.base_config.Config'>. To extend the existing keys, use `override` with `is_strict` = False."

In [147]:
from official.core.config_definitions import base_config
task = RetinaNetTask(base_config.Config(model_yaml))


TypeError: 'map' object does not support item assignment

In [144]:
task.build_model()

AttributeError: 'dict' object has no attribute 'model'

In [121]:
from official.vision.configs.retinanet import RetinaNet

In [130]:

RetinaNet(**(model_yaml["task"]["model"]))

TypeError: 'map' object does not support item assignment

In [197]:
from official.core.config_definitions import ExperimentConfig, TaskConfig, TrainerConfig, RuntimeConfig
from official.vision.configs import retinanet
from official.vision.tasks import RetinaNetTask
import dataclasses
@dataclasses.dataclass
class RetinanetExperimentConfig(base_config.Config):
    task: TaskConfig = retinanet.RetinaNetTask()
    trainer: TrainerConfig = TrainerConfig()
    runtime: RuntimeConfig = RuntimeConfig()

In [198]:
cfg = RetinanetExperimentConfig.from_yaml(pipeline_config)

In [168]:
cf= RetinaNetTask(cfg.task)

In [170]:
mdl = cf.build_model()

In [174]:
cf.initialize(mdl)

In [176]:
mdl.predict(np.zeros((1,384,384,3)))

ValueError: in user code:

    File "C:\Users\maxime.carrere\PycharmProjects\xplique\venv\lib\site-packages\keras\engine\training.py", line 2041, in predict_function  *
        return step_function(self, iterator)
    File "C:\Users\maxime.carrere\PycharmProjects\xplique\venv\lib\site-packages\keras\engine\training.py", line 2027, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\maxime.carrere\PycharmProjects\xplique\venv\lib\site-packages\keras\engine\training.py", line 2015, in run_step  **
        outputs = model.predict_step(data)
    File "C:\Users\maxime.carrere\PycharmProjects\xplique\venv\lib\site-packages\keras\engine\training.py", line 1983, in predict_step
        return self(x, training=False)
    File "C:\Users\maxime.carrere\PycharmProjects\xplique\venv\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\MAXIME~1.

In [233]:
from official.vision.serving import detection
cfg.task.model.num_classes = 91
cfg.task.init_checkpoint = "./models/ckpt-277200"
det = detection.DetectionModule(cfg, batch_size=1, input_image_size=[384, 384])

In [234]:
#wget -q 'https://unsplash.com/photos/XOxsVJmCyxk/download?force=true&w=640' -O 'fox.jpg'

x = np.expand_dims(tf.keras.preprocessing.image.load_img("james-resly-XOxsVJmCyxk-unsplash.jpg", target_size=(384, 384)), 0)
x = np.array(x, dtype=np.float32) #/ 255.0

In [225]:
det.serve(x)

{'detection_boxes': <tf.Tensor: shape=(1, 100, 4), dtype=float32, numpy=
 array([[[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         

In [189]:
tfm.vision.configs.retinanet.DetectionGenerator

official.vision.configs.retinanet.DetectionGenerator

In [212]:
x.shape

(1, 384, 384, 3)

In [231]:
det.model.load_weights("models/ckpt-277200")

AssertionError: Nothing except the root object matched a checkpointed value. Typically this means that the checkpoint does not match the Python program. The following objects have no matching checkpointed value: [<tf.Variable 'sync_batch_normalization_855/moving_variance:0' shape=(24,) dtype=float32, numpy=
array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1.], dtype=float32)>, <tf.Variable 'inverted_bottleneck_block_244/squeeze_excitation/conv2d_3/kernel:0' shape=(1, 1, 22, 672) dtype=float32, numpy=
array([[[[-0.00676957, -0.09634139, -0.18770218, ...,  0.19944258,
          -0.14135365,  0.18210804],
         [-0.22393149,  0.19635543,  0.3284623 , ...,  0.00707996,
           0.17528068,  0.10661352],
         [-0.06039536,  0.03249402, -0.04811416, ..., -0.02716474,
          -0.34512892,  0.44819278],
         ...,
         [ 0.1813063 , -0.062728  ,  0.04288131, ...,  0.22318785,
           0.03546751, -0.18583813],
       

In [235]:
ckpt = tf.compat.v2.train.Checkpoint(model=det.model)
ckpt.restore("models/ckpt-277200").expect_partial()

ValueError: Received incompatible tensor with shape (80,) when attempting to restore variable with shape (112,) and name backbone/layer_with_weights-100/beta/.ATTRIBUTES/VARIABLE_VALUE.

In [228]:
task = tfm.core.task_factory.get_task(cfg.task)
model = task.build_model()
task.initialize(model)

ValueError: Received incompatible tensor with shape (80,) when attempting to restore variable with shape (112,) and name backbone/layer_with_weights-100/beta/.ATTRIBUTES/VARIABLE_VALUE.